In [ ]:
#%reset -f -> clears the global namespace without user confirmation.
%reset -f

#from init import *
#from main_functions import *
#from utility_functions import *
#%load main_functions.py
%run init.py
%run main_functions.py
%run utility_functions.py

'''
CONFIGURATION SETUP
'''
SEPARATE_SENT = True
SHOW_DP_PLOTS = True
SHOW_REL_EXTRACTIONS = True
NODE_SELECTION = True
MAX_ITERATION = 100 # -1 -> to try all
SAVE_GEFX = True
SAVE_PAIRWISE_RELS = True
SAVE_ALL_RELS = True
SET_INOUT_FNAMES_IN_CODE_ARGS = False


'''
INPUT/OUTPUT SETUP - PASS IT AS ARGUMENTS - PARAMETERS
'''
data_dir = "../../data/"
DATA_SET = "mothering"
texts = []
#output_dir = ""

if SET_INOUT_FNAMES_IN_CODE_ARGS:
    file_input = str(sys.argv[1])
    output_dir = str(sys.argv[2])
    file_input_name = os.path.basename(file_input)
    file_input_name = str(file_input_name.split(".")[0])    
else:
    if DATA_SET == "twitter":
        based_dir = data_dir + 'Tweets/'
        output_dir = based_dir  + 'output/'      
        file_input_name = 'sample.csv'
        file_input = based_dir + file_input_name      
        df = read_data(file_input,"twitter",",")#read the input sentences
        texts = df['main_tweet'].tolist()
    elif DATA_SET == "mothering":
        based_dir = data_dir + 'Vaccination/'
        output_dir = based_dir + 'output/'
        file_input_name = 'sents.txt'
        #file_input_name = 'sent_cdb_child_exemption.txt'
        file_input = based_dir + file_input_name
        df = read_data(file_input,"mothering","\n")#read the input sentences
        texts = df['text'].tolist()
    elif DATA_SET == "test":
        based_dir = data_dir + 'Vaccination/'
        output_dir = based_dir + 'output/'
        file_input_name = "religousInst_adverseEffects.csv"
        df = read_data(data_dir + "Vaccination/mothering_contexts/" + file_input_name ,"mothering","\n")
        texts = df['sentence'].tolist()
        print len(df)
        

f_rel = open(output_dir + file_input_name + "_" + "relations_" + str(MAX_ITERATION) + ".csv" , "w")
header = ['original_text', 'sentence','arg1','rel','arg2','type','pattern','arg1_with_pos','rel_with_pos','arg2_with_pos','arg1_prepositions', 'rel_prepositions', 'arg2_prepositions']
dict_writer = csv.DictWriter(f_rel, header)
dict_writer.writeheader()

'''
A few sample test cases:
#texts = ["the Church told all Catholic parents not to let their child get the MMR."]
#texts = ["Why Samsung Pay could gain an early lead in mobile payments."]
#texts = ["You would keep your child 's shot records at home and NOT submit that to the school...only your exemption from all shots ."]
#texts = ["Parents may use their philosophical beliefs exemption for ANY vaccine they choose to do so ; you may selectively vaccinate your child and exempt them out of other vaccines ; you may also exempt out of any and all vaccines and use your exemption that way , as well ."]
#texts = ["Here is the Hawaii immunization brochure , which states the exemption forms can also be obtained from the school : Immunization and TB code : Surprisingly , I do n't see anything about religiously exempting a child from the TB screening requirement in the code ."]
#texts.insert(0,"I like this product and I also like the other product.") ! Strange dep res -> prep_like
#texts.insert(0,"A medical exemption is out of the question - you 'd have to first find a doctor willing to exempt your child from all shots - not going to happen - and then have the medical exemption renewed every year . ") -> tokenizer failure
#texts.insert(0,"A state exemption is only to exempt a child from state requirements ; while in the states , the only time an exemption would come up is when using DODs schools or daycare in the states--the OP is going to Japan . ")
#texts.insert(0,"Even if the Church told all Catholic parents not to let their child get the MMR for instance , most parents would have to still be required to submit a religous exemption which would exempt all vaccines .")
'''
texts.insert(0,"parents get medical exemption from vaccination for their child.")
texts.insert(0,"Parents get medical exemption from vaccination for their child.")
texts.insert(0,"Schools don't accept children with no medical exemptions.")

annotator = Annotator()
# string versions of all relations - used in aggregation matching
all_rels_str = []
# dataframe of relations
all_rels = []
# the final output kept in csv format. Includes, original post, sentence, and their extractions/tags.
output = []

# To keep track of running time 
start_time = time.time()

for ind, t_orig in enumerate(texts):
    if MAX_ITERATION >= 0:
        if ind > MAX_ITERATION:
            break
    t_sentences = []
    
    try:
        # replacing - with space -> since most of the time tokenazer fails with the presence of -
        #t_orig = t_orig.replace("-",".")
        #t_orig = t_orig.replace(" don't ", " do not ")
        #t_orig = change_nt_to_not(t_orig)
        # tokenizing the sentence
        #t_sentences = sent_tokenize(t_orig)
        
        
        if CLEAN_SENTENCES:
            t_orig = clean_sent(t_orig)
        if SEPARATE_SENT:
            t_sentences = sent_tokenize(t_orig)
        else:
            t_sentences = [t_orig]        
    except:
        print "Error in sentence tokenizer! - ", t_orig
        
    
    for t in t_sentences:
        try:
            t_annotated = annotator.getAnnotations(t, dep_parse=True)
        except:
            print "Error in sentence annotation"
            continue
        try:
            g_dir = create_dep_graph(t_annotated)
            if g_dir is None:
                print "No extraction found"
                continue
            if SHOW_DP_PLOTS:
                plot_dep(g_dir,t)
            g_undir = g_dir.to_undirected()
        except:
            print "Unexpected error while extracting relations:", sys.exc_info()[0]
            continue
        rels_pure, rels_simp = get_relations(g_dir, t_annotated, option="SVO")
        rels = rels_pure#rels_simp
        if SHOW_REL_EXTRACTIONS:
            print ind, t, "\n"
            print "Simplifided Version:"
            print_relations(rels)
            print "More detailed Version:"
            print_relations(rels_pure)
        else:
            print ind,
        all_rels_str = all_rels_str + get_rels_str(rels) #For simply counting the exact strings
        all_rels = all_rels + rels # to later create a dataframe
        for r in rels:
            output_row = defaultdict(list)
            output_row = r.copy()
            output_row["original_text"] = t_orig
            output_row["sentence"] = t
            output.append(output_row)
            #print " output is : ", output
            #output_subset = dict((k,output[k]) for k in header)
            dict_writer.writerow(output_row)


end_time = time.time()
print "Relation Extraction Time: ", end_time-start_time , "(seconds) - ", (end_time-start_time)/60, "(min)"
print "***************STATISTICS***************"
print "Total number of input records (posts): ", len(texts)
print "Total number of extracted relations: ", len(all_rels_str)
print_top_relations(all_rels_str,top_num=-1) 

df_rels = pd.DataFrame(all_rels)
df_output = pd.DataFrame(output)
print df_output

if SAVE_ALL_RELS:
    columns = ['original_text', 'sentence','arg1','rel','arg2','type','pattern','arg1_with_pos','rel_with_pos','arg2_with_pos', 'arg1_prepositions', 'rel_prepositions', 'arg2_prepositions']
    df_output.to_csv(output_dir + file_input_name + "_" + "output_relations_test.csv",sep=',', encoding='utf-8',header=True, columns=columns)
    #save_df_rels(df_rels)

In [ ]:
#'''
if NODE_SELECTION:
    # get the list of different versions of an entity. Example : parents,parent,i,we -> parents
    entity_versions = get_entity_versions(DATA_SET)    
    df_simp = get_simp_df(df_rels.copy(),entity_versions)  
    selected_nodes = entity_versions.keys()
    df_rels_selected = filter_nodes(df_simp.copy(),source='arg1',target='arg2',selected_nodes = selected_nodes)
    g_arg = create_argument_multiGraph(df_rels_selected.copy(),source='arg1',target='arg2',edge_attr = 'rel')
    if SAVE_GEFX:
        nx.write_gexf(g_arg, output_dir + file_input_name + "_" + "g_arg_selected_"+str(MAX_ITERATION)+"_"+str(time.time())+".gexf")
    plot_argument_graph(g_arg)
    if SAVE_PAIRWISE_RELS:
        file_loc = output_dir + file_input_name + "_" + "pairwise_rels_selected_"+str(MAX_ITERATION)+"_"+DATA_SET+".txt"
        save_pairwise_rels(file_loc,g_arg,print_option=True)      

g_arg = create_argument_multiGraph(df_rels.copy(),source='arg1',target='arg2',edge_attr = 'rel')
if SAVE_GEFX:
    nx.write_gexf(g_arg, output_dir + file_input_name + "_" + "g_arg_"+str(MAX_ITERATION)+"_"+str(time.time())+".gexf")
plot_argument_graph(g_arg)
if SAVE_PAIRWISE_RELS:
    file_loc = output_dir + file_input_name + "_"  + "pairwise_rels_"+str(MAX_ITERATION)+"_"+DATA_SET+".txt"
    save_pairwise_rels(file_loc,g_arg,print_option=True)    
    
#if __name__ == "__main__":
#    main(sys.argv[1:])
#'''

In [ ]:
#'''
# In[68]:

entity = "government"
df_simp[np.logical_and(df_simp['arg1']==entity,df_simp['arg2']==entity)]


# In[74]:

selected_nodes = entity_versions.keys()
df_rels_selected = filter_nodes(df_simp.copy(),source='arg1',target='arg2',selected_nodes = selected_nodes)
g_arg = create_argument_multiGraph(df_rels_selected.copy(),source='arg1',target='arg2',edge_attr = 'rel')
if SAVE_GEFX:
    nx.write_gexf(g_arg, output_dir +  "/gephi_data/g_arg_selected_2_"+str(MAX_ITERATION)+"_"+str(time.time())+".gexf")
plot_argument_graph(g_arg)
if SAVE_PAIRWISE_RELS:
    file_loc = output_dir + "/pairwise_rels_selected_2_"+str(MAX_ITERATION)+"_"+DATA_SET+".txt"
    save_pairwise_rels(file_loc,g_arg,print_option=True)  


# In[72]:

df_rels_selected

In [9]:
based_dir = data_dir + 'Vaccination/'
output_dir = based_dir + 'output/'
file_input_name = 'allsentrels.csv'
file_input = based_dir + file_input_name
df2 = pd.read_csv(file_input,delimiter=',',header=0,error_bad_lines=False)   

In [16]:
entity_versions = get_entity_versions(DATA_SET)

In [18]:
df_simp = get_simp_df(df2.copy(),entity_versions)  
selected_nodes = entity_versions.keys()
#selected_nodes = ['medical prof', 'vaccines']
df2_selected = filter_nodes(df_simp.copy(),source='arg1',target='arg2',selected_nodes = selected_nodes)

AttributeError: 'NoneType' object has no attribute 'group'

In [14]:
entity_versions.keys()

['VPDs',
 'medical prof',
 'government',
 'religous inst',
 'vaccines',
 'parents',
 'adverse effects',
 'schools',
 'children',
 'exemptions']

In [12]:
g_arg = create_argument_multiGraph(df2.copy(),source='arg1',target='arg2',edge_attr = 'rel')

KeyboardInterrupt: 

In [ ]:
if SAVE_GEFX:
    nx.write_gexf(g_arg, output_dir + file_input_name + "_" + "g_arg_all2_"+str(MAX_ITERATION)+"_"+str(time.time())+".gexf")
plot_argument_graph(g_arg)
if SAVE_PAIRWISE_RELS:
    file_loc = output_dir + file_input_name + "_" + "pairwise_rels_all2_"+str(MAX_ITERATION)+"_"+DATA_SET+".txt"
    save_pairwise_rels(file_loc,g_arg,print_option=True)  

In [27]:
df3 = df2[df2["sentence"].str.contains("I was telling")]
df3

,sentence,arg1,rel,arg2
28158,After our last visit I was telling him how ups...,last {visit},was {telling},{him}
28159,After our last visit I was telling him how ups...,{I},was {telling},{him}
28160,After our last visit I was telling him how ups...,{she},{doesn},{'t}
28614,After our last visit I was telling him how ups...,last {visit},was {telling},{him}
28615,After our last visit I was telling him how ups...,{I},was {telling},{him}
28616,After our last visit I was telling him how ups...,{she},{doesn},{'t}
32552,I was telling the dr how she had the DTaP that...,{I},was {telling},the {dr}
33074,I was telling the dr how she had the DTaP that...,{I},was {telling},the {dr}
70969,I was telling him that I didn't think she woul...,{I},was {telling},{him}
71575,I was telling him that I didn't think she woul...,{I},was {telling},{him}


KeyError: 0

In [51]:
based_dir = data_dir + 'Vaccination/'
output_dir = based_dir + 'output/'
#file_input_name = 'sents.txt'
#file_input_name = 'sent_cdb_child_exemption.txt'
file_input = based_dir + file_input_name
df = read_data(file_input,"mothering","\n")#read the input sentences
texts = df['text'].tolist()

Skipping line 44759: expected 1 fields, saw 2



In [52]:
print len(texts)

26837


In [48]:
sent = "parents get medical exemption from vaccination for their child."
print help(annotator.getDependency)

Help on method getDependency in module practnlptools.tools:

getDependency(self, parse) method of practnlptools.tools.Annotator instance

None


In [45]:
dep

''